# PoC 3: Excel ログ読み込みと突合テスト

src/ 配下のクリーンアーキ構成を通じて、

1. Excel から操作履歴・状態変化を読み込む
2. ContractId でフィルタ（任意）
3. 時間突合してフラグ付け
4. Parquet/CSV に保存

を試すためのノートブックです。

In [ ]:
# 依存ライブラリ
import os
import pandas as pd

from src.interface.notebook import load_for_notebook
from src.domain.models import build_merged_dataset

# 環境変数で突合許容差をセット（必要に応じて変更）
os.environ.setdefault("MERGE_TOLERANCE_MINUTES", "5")

# ContractId でフィルタしたい場合はリストを指定、全件なら None
CONTRACT_FILTER = "C19282010202200078"  # 例: ["C123", "C456"]


In [ ]:
# 1. Excel読み込み（キャッシュ利用）
op_df, state_df = load_for_notebook(contract_ids=CONTRACT_FILTER)
display(op_df.head())
display(state_df.head())
print(f"操作件数: {len(op_df):,}, 状態変化件数: {len(state_df):,}")

In [ ]:
# 2. 突合して分析用データセットを作成
merged = build_merged_dataset(op_df, state_df)
df = merged.df
display(df.head())
print(f"突合後件数: {len(df):,}")
print(f"is_remote_operation=True 件数: {df['is_remote_operation'].sum():,}")

In [ ]:
# 3. 保存先を指定して Parquet / CSV を出力（必要に応じてコメントアウト）
OUTPUT_PARQUET = "data/merged_logs.parquet"
OUTPUT_CSV = "data/merged_logs.csv"

merged.to_parquet(OUTPUT_PARQUET)
merged.to_csv(OUTPUT_CSV)
print(f"Saved: {OUTPUT_PARQUET}, {OUTPUT_CSV}")